In [10]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from datetime import datetime

In [11]:
dtypes = {
    'aggregate': 'object',
    'cases': 'float64',
    'city': 'object',
    'population': 'float64',
    'country': 'object',
    'county': 'object',
    'level': 'object',
    'state': 'object',
    'deaths': 'float64'
}

In [12]:
df = dd.read_csv('timeseries.csv', dtype=dtypes)
print(df.dtypes)

name                    string[pyarrow]
level                   string[pyarrow]
city                    string[pyarrow]
county                  string[pyarrow]
state                   string[pyarrow]
country                 string[pyarrow]
population                      float64
lat                             float64
long                            float64
url                     string[pyarrow]
aggregate               string[pyarrow]
tz                      string[pyarrow]
cases                           float64
deaths                          float64
recovered                       float64
active                          float64
tested                          float64
hospitalized                    float64
hospitalized_current            float64
discharged                      float64
icu                             float64
icu_current                     float64
growthFactor                    float64
date                    string[pyarrow]
dtype: object


In [13]:
df['date'] = dd.to_datetime(df['date'])
mask = (
    (df['country'] == 'United States') & 
    (df['level'] == 'state') &
    (df['date'] >= '2020-01-01') & 
    (df['date'] <= '2021-02-28')
)
us_states = df[mask].compute() 

In [15]:
state_mortality = us_states.groupby('state').agg({
    'deaths': 'sum',
    'population': 'mean'
})

state_mortality['per_capita_mortality'] = state_mortality['deaths'] / state_mortality['population']
mortality_ranking = state_mortality.sort_values('per_capita_mortality', ascending=False)

print(mortality_ranking)

                                 deaths  population  per_capita_mortality
state                                                                    
New Jersey                     861317.0   8882190.0              0.096971
Connecticut                    277777.0   3565287.0              0.077912
Massachusetts                  482980.0   6892503.0              0.070073
New York                      1358824.0  19453561.0              0.069850
Louisiana                      213399.0   4648794.0              0.045904
Washington, D.C.                32225.0    705749.0              0.045661
Rhode Island                    46435.0   1059361.0              0.043833
Michigan                       419841.0   9986857.0              0.042039
Illinois                       382919.0  12671821.0              0.030218
Pennsylvania                   378963.0  12801989.0              0.029602
Maryland                       175443.0   6045680.0              0.029020
Delaware                        25878.

In [16]:
# Using WHO approach: deaths divided by cases (simple CFR)
monthly_data = us_states.groupby(['state', pd.Grouper(key='date', freq='M')]).agg({
    'deaths': 'sum',
    'cases': 'sum'
}).reset_index()

cfr_matrix = pd.pivot_table(
    monthly_data,
    values='deaths',
    index='state',
    columns=monthly_data['date'].dt.strftime('%Y-%m'),
    aggfunc='sum'
) / pd.pivot_table(
    monthly_data,
    values='cases',
    index='state',
    columns=monthly_data['date'].dt.strftime('%Y-%m'),
    aggfunc='sum'
)

print(cfr_matrix)

date                          2020-01   2020-02   2020-03   2020-04   2020-05  \
state                                                                           
Alabama                           NaN       NaN  0.005324  0.029829  0.038967   
Alaska                            NaN       NaN  0.000000  0.026795  0.024091   
American Samoa                    NaN       NaN       NaN       NaN       NaN   
Arizona                           0.0  0.000000  0.000000  0.029733  0.039077   
Arkansas                          NaN       NaN  0.009231  0.018736  0.021011   
California                        0.0  0.000000  0.020168  0.034800  0.039834   
Colorado                          NaN       NaN  0.018294  0.043694  0.053447   
Connecticut                       NaN       NaN  0.018184  0.064257  0.089880   
Delaware                          NaN       NaN  0.013341  0.027265  0.036570   
Florida                           NaN       NaN  0.008534  0.029055  0.042748   
Georgia                     

C:\Users\91849\AppData\Local\Temp\ipykernel_2656\486204788.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = us_states.groupby(['state', pd.Grouper(key='date', freq='M')]).agg({


In [17]:
cfr_changes = cfr_matrix.diff(axis=1)

# Aggregate changes (sum of absolute changes)
total_cfr_change = cfr_changes.abs().sum(axis=1)
cfr_change_ranking = total_cfr_change.sort_values(ascending=False)

print(cfr_change_ranking)

state
Arizona                         0.089165
New Jersey                      0.083710
Michigan                        0.079918
Northern Mariana Islands        0.079321
Connecticut                     0.075907
Massachusetts                   0.074312
New York                        0.072444
Washington                      0.068538
Wisconsin                       0.065754
Pennsylvania                    0.064987
New Hampshire                   0.061563
Minnesota                       0.061049
Missouri                        0.056718
Florida                         0.056576
Oklahoma                        0.055996
United States Virgin Islands    0.055598
Rhode Island                    0.055043
Oregon                          0.054662
California                      0.054237
Nevada                          0.053798
Kansas                          0.053087
Ohio                            0.052443
West Virginia                   0.052068
Louisiana                       0.050801
Maryland  